### Lab Tasks
- In the dataframe creates in Lab 2 - Part a set ``Salary`` as the target value. 
- The rest of the columns are considered as X, feature set. 
- Use ``train_test_split`` to split the dataset into train and test dataset. set ``random_state = 0``.
- Use ``MinMaxScaler`` to scale feature set X. 

### Lab Questions

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('adult.csv')

In [2]:
data = data[data != ' ?']

C:\Users\obaid\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [3]:
l = ['workclass', 'education', 'occupation', 'native-country']
data.drop(l, axis = 1, inplace = True)

In [4]:
df = pd.get_dummies(data['marital-status'])
data = pd.concat([data, df], axis = 1)
data.drop('marital-status', axis = 1, inplace=True)

In [5]:
df = pd.get_dummies(data['relationship'])
data = pd.concat([data, df], axis = 1)
data.drop('relationship', axis = 1, inplace=True)

In [6]:
df = pd.get_dummies(data['race'])
data = pd.concat([data, df], axis = 1)
data.drop('race', axis = 1, inplace=True)

In [7]:
data['sex'] = data['sex'].map({' Male':0, ' Female':1})

In [8]:
data['Salary'] = data['Salary'].map({' <=50K':0, ' >50K':1})

In [9]:
data.columns

Index(['age', 'fnlwgt', 'education-num', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'Salary', ' Divorced', ' Married-AF-spouse',
       ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married',
       ' Separated', ' Widowed', ' Husband', ' Not-in-family',
       ' Other-relative', ' Own-child', ' Unmarried', ' Wife',
       ' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other',
       ' White'],
      dtype='object')

#### Question 1:
What is the ratio of label 1 to label 0? (consider two significant digits)

In [10]:
print(data['Salary'].count())

oneCount = data[ (data['Salary'] ==1)]['Salary'].count()
zeroCount = data[ (data['Salary'] ==0)]['Salary'].count()

print(oneCount ,"oneCount")
print(zeroCount,"zeroCount")

ratio = oneCount/zeroCount
print(ratio)

32561
7841 oneCount
24720 zeroCount
0.31719255663430423


#### Question 2: 
Consider the following scenario:
Government is securing a budget for low-income assistance programs. Assume this program will help low-income individuals/families with a fixed rate financial support. We need to have a good model to predict how many individuals/families are eligible for this assistance program. Beside accuracy which evaluation model should we use? 

```Python
' <=50K':0 - Negative
' >50K':1  - Positive```

## Answer is PRECISION as we are intersted in having a cery accurate True Positive. Precision is TP/(TP+FP) 

In [11]:
y= data['Salary']
X = data.drop(['Salary'],axis =1)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x_train_org,x_test_org,y_train,y_test=train_test_split(X,y,random_state=0)  ## org stands for the very original
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train_org)  ### you can fit and transform together 
X_test = scaler.transform(x_test_org)  

C:\Users\obaid\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


#### Question 3 and 4:    l1 is lasso regularization
Train a ``LogisticRegression`` model on this dataset. Set the hyper-parameters as follow: 
```Python
penalty = 'l1'
random_state = 0```
What is the train and test precision score? (with two significant digits) 

In [12]:

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty='l1',random_state=0,)
logreg.fit(X_train,y_train)

print("Train accuracy ",logreg.score(X_train,y_train))
print("Test accuracy",logreg.score(X_test,y_test))

from sklearn.metrics import precision_score

print("TRAIN precision score: ",precision_score(y_train,logreg.predict(X_train)))
print("Test Precision score:",precision_score(y_test,logreg.predict(X_test)))

C:\Users\obaid\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train accuracy  0.8436117936117936
Test accuracy 0.8454735290504852
TRAIN precision score:  0.7208748375920312
Test Precision score: 0.7271016311166876


#### Question 5 and 6:
Train a dummy classifier that classifies everything as the majority class of the training data. What are the train and test accuracy of this classifier? (numbers with two significant digits)

In [13]:
from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)



print("train accuracy score : {}".format(dummy_majority.score(X_train,y_train)) )
print("test accuracy score : {}".format(dummy_majority.score(X_test,y_test)) )

train accuracy score : 0.76007371007371
test accuracy score : 0.7565409654833558


#### Questions 7:
Train a ``LogisticRegression`` model on this dataset. Set the hyper-parameters as follow: 
```Python
penalty = 'l1'
random_state = 0``` 
What is the precision score when using a threshold of -2 on the decision function. Use ``X_test`` and ``y_test``. (2 significant digits)

In [14]:


thresh_val = 2
dec_scores = logreg.decision_function(X_test)
newPred = np.where( dec_scores > thresh_val, 1, 0)

print("Test Precision score at new thresh value:",precision_score(y_test,newPred))

Test Precision score at new thresh value: 0.9410029498525073


In [37]:
X_test = pd.DataFrame(X_test)

In [39]:
X_test['dec_scores']= logreg.decision_function(X_test)

In [46]:
X_test = X_test.sort_values(by=['dec_scores'],ascending=False)

In [51]:
best_customers = X_test[0:500]   ### best 1000 customers 

In [52]:
best_customers

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,dec_scores
3105,0.479452,0.218679,1.000000,0.0,1.000000,0.000000,0.653061,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.739250
6997,0.547945,0.099661,0.933333,0.0,1.000000,0.000000,0.602041,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.260235
5786,0.315068,0.143706,0.933333,0.0,1.000000,0.000000,0.704082,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.220186
4181,0.328767,0.209654,0.933333,0.0,1.000000,0.000000,0.653061,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.129985
4118,0.356164,0.305548,0.933333,0.0,1.000000,0.000000,0.602041,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.083710
6050,0.301370,0.120898,1.000000,0.0,1.000000,0.000000,0.551020,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,32.053906
7653,0.397260,0.073244,0.933333,0.0,1.000000,0.000000,0.602041,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.989549
1427,0.657534,0.134538,0.800000,0.0,1.000000,0.000000,0.653061,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.925907
5932,0.123288,0.165520,0.933333,0.0,1.000000,0.000000,0.602041,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.595800
132,0.273973,0.237552,0.933333,0.0,1.000000,0.000000,0.500000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.579721


In [53]:
best_customers['dec_scores'].describe()

count    500.000000
mean       5.299321
std        7.561318
min        1.497904
25%        1.869888
50%        2.710166
75%        4.910539
max       32.739250
Name: dec_scores, dtype: float64